In [28]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

C:\Users\user\AppData\Local\Temp\ipykernel_19144\3510566465.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [27]:
import os
import shutil

#베이스라인 모델 학습을 위한 준비
import torch

USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

BATCH_SIZE =256
EPOCH=30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from torch.utils.data import DataLoader

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],
                            [0.229,0.224,0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize([64,64]),
        transforms.RandomCrop(52),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
}

data_dir='C:/Users/user/Desktop/kang/spliteed'
image_datasets= {x:ImageFolder(root=os.path.join(data_dir,x), transform=data_transforms[x]) for x in ['train','val']}
dataloaders= {x:torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train','val']}
dataset_sizes= {x: len(image_datasets[x]) for x in ['train','val']}
class_names=image_datasets['train'].classes

cuda


In [9]:
#Pre-Trained Model 불러오기
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

resnet = models.resnet50(pretrained=True)
num_ftrs=resnet.fc.in_features
resnet.fc=nn.Linear(num_ftrs,33)
resnet=resnet.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer_ft=optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)

from torch.optim import lr_scheduler

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [10]:
#Pre-Trained Model 일부 Layer Freeze 하기
ct=0
for child in resnet.children():
    ct+=1
    if ct<6:
        for param in child.parameters():
            param.requires_grad=False

In [21]:
#Transfer Learning 모델 학습과 검증을 위한 함수
import time
import copy

def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):
    
    best_mode_wts=copy.deepcopy(model.state_dict())
    best_acc=0.0
    
    for epoch in range(num_epochs):
        print('epoch {}'.format(epoch+1))
        since=time.time()
        
        for phase in ['train','val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss=0.0
            running_corrects=0
            
            for inputs, labels in dataloaders[phase]:
                inputs= inputs.to(DEVICE)
                labels= labels.to(DEVICE)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds= torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase=='train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
                l_r=[x['lr'] for x in optimizer_ft.param_groups]
                print('learning rate:', l_r)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc= running_corrects.double()/dataset_sizes[phase]
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase=='val' and epoch_acc > best_acc :
                best_acc=epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        time_elapsed =time.time()-since
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed//60, time_elapsed % 60))
        
    model.load_state_dict(best_model_wts)
        
    return model

In [22]:
#모델 학습 실행하기
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH)
torch.save(model_resnet50, 'resnet50.pt')

epoch 1
learning rate: [0.001]
train Loss: 0.1660 Acc: 0.9466
val Loss: 0.1411 Acc: 0.9532
Completed in 0m 50s
epoch 2
learning rate: [0.001]
train Loss: 0.1301 Acc: 0.9583
val Loss: 0.1590 Acc: 0.9527
Completed in 0m 50s
epoch 3
learning rate: [0.001]
train Loss: 0.1151 Acc: 0.9619
val Loss: 0.1565 Acc: 0.9483
Completed in 0m 51s
epoch 4
learning rate: [0.001]
train Loss: 0.1020 Acc: 0.9677
val Loss: 0.1492 Acc: 0.9544
Completed in 0m 50s
epoch 5
learning rate: [0.0001]
train Loss: 0.0952 Acc: 0.9690
val Loss: 0.1324 Acc: 0.9574
Completed in 0m 50s
epoch 6
learning rate: [0.0001]
train Loss: 0.0503 Acc: 0.9840
val Loss: 0.0451 Acc: 0.9861
Completed in 0m 50s
epoch 7
learning rate: [0.0001]
train Loss: 0.0332 Acc: 0.9898
val Loss: 0.0434 Acc: 0.9854
Completed in 0m 50s
epoch 8
learning rate: [0.0001]
train Loss: 0.0259 Acc: 0.9921
val Loss: 0.0366 Acc: 0.9889
Completed in 0m 51s
epoch 9
learning rate: [0.0001]
train Loss: 0.0243 Acc: 0.9917
val Loss: 0.0392 Acc: 0.9886
Completed in 0m 